<a href="https://colab.research.google.com/github/tanriders/Recommend_movies/blob/main/DS_Marketing_Recommend_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **ระบบแนะนำภาพยนต์**

ระบบแนะนำภาพยนต์ โดยใช้ความคล้ายของ keyword หรือ Meta-data description ของภาพยนต์

หลักการคำนวณความคล้ายใช้ค่า Cosine Similarity

import และโหลดไฟล์ข้อมูลภาพยนต์และตรวจสอบเบื้องต้น

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("/content/drive/MyDrive/DS/DS-Marketing/movie.csv")

In [ ]:
df.shape

(4803, 24)

มีข้อมูลจำนวน 4803 แถว 24 คอลัมน์ (ภาพยนต์จำนวน 4803 เรื่อง)

In [ ]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


ตรวจดูข้อมูลที่สูญหาย

In [ ]:
df.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

พบว่ามีข้อมูลหายจำนวนมาก ซึ่งบางคอลัมน์อาจไม่จำเป็นต้องใช้

In [ ]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [ ]:
df[df.title.str.contains('transformer',case=False)].title

35     Transformers: Revenge of the Fallen
36         Transformers: Age of Extinction
52          Transformers: Dark of the Moon
111                           Transformers
Name: title, dtype: object

In [ ]:
df.iloc[111].overview

'Young teenager, Sam Witwicky becomes involved in the ancient struggle between two extraterrestrial factions of transforming robots – the heroic Autobots and the evil Decepticons. Sam holds the clue to unimaginable power and the Decepticons will stop at nothing to retrieve it.'

In [ ]:
df.iloc[111].keywords

'destroy transformation alien based on toy transformers'

In [ ]:
ids = [35,36,52]
for k in ids:
  print('\n\t Title: ', df.iloc[k].title)
  print('overview: ', df.iloc[k].overview)
  print()
  print('kwd: ',df.iloc[k].keywords)
  print('----------------------------')


	 Title:  Transformers: Revenge of the Fallen
overview:  Sam Witwicky leaves the Autobots behind for a normal life. But when his mind is filled with cryptic symbols, the Decepticons target him and he is dragged back into the Transformers' war.

kwd:  egypt sun chaos symbol artifact
----------------------------

	 Title:  Transformers: Age of Extinction
overview:  As humanity picks up the pieces, following the conclusion of "Transformers: Dark of the Moon," Autobots and Decepticons have all but vanished from the face of the planet. However, a group of powerful, ingenious businessman and scientists attempt to learn from past Transformer incursions and push the boundaries of technology beyond what they can control - all while an ancient, powerful Transformer menace sets Earth in his cross-hairs.

kwd:  sequel alien transformers giant robot robot
----------------------------

	 Title:  Transformers: Dark of the Moon
overview:  Sam Witwicky takes his first tenuous steps into adulthood whil

In [ ]:
df.iloc[0:2,3:8]

,homepage,id,keywords,original_language,original_title
0,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar
1,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End


In [ ]:
for k in df.columns:
  print(k,': ', df[k].iloc[111])
  print('--------------')

index :  111
--------------
budget :  150000000
--------------
genres :  Adventure Science Fiction Action
--------------
homepage :  http://www.transformersmovie.com/
--------------
id :  1858
--------------
keywords :  destroy transformation alien based on toy transformers
--------------
original_language :  en
--------------
original_title :  Transformers
--------------
overview :  Young teenager, Sam Witwicky becomes involved in the ancient struggle between two extraterrestrial factions of transforming robots – the heroic Autobots and the evil Decepticons. Sam holds the clue to unimaginable power and the Decepticons will stop at nothing to retrieve it.
--------------
popularity :  25.468493
--------------
production_companies :  [{"name": "Paramount Pictures", "id": 4}, {"name": "DreamWorks SKG", "id": 27}, {"name": "Amblin Entertainment", "id": 56}, {"name": "Di Bonaventura Pictures", "id": 435}, {"name": "Platinum Dunes", "id": 2481}, {"name": "thinkfilm", "id": 12247}, {"name": "

In [ ]:
features = ['keywords','cast','genres','director']
df[features].isnull().sum()

keywords    412
cast         43
genres       28
director     30
dtype: int64

In [ ]:
features = ['keywords','cast','genres','director']
for feature in features:
  df[feature] = df[feature].fillna("")


In [ ]:
df[features].isnull().sum()

keywords    0
cast        0
genres      0
director    0
dtype: int64

In [ ]:
df['combined_features'] = df.genres + ' ' + df.keywords + ' ' + df.cast + ' ' + df.director
df.combined_features.head()

0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
Name: combined_features, dtype: object

In [ ]:
for i in range(0,3):
  print(df.title[i])
  print(df.combined_features[i])
  print('----------------')

Avatar
Action Adventure Fantasy Science Fiction culture clash future space war space colony society Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez James Cameron
----------------
Pirates of the Caribbean: At World's End
Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life Johnny Depp Orlando Bloom Keira Knightley Stellan Skarsg\u00e5rd Chow Yun-fat Gore Verbinski
----------------
Spectre
Action Adventure Crime spy based on novel secret agent sequel mi6 Daniel Craig Christoph Waltz L\u00e9a Seydoux Ralph Fiennes Monica Bellucci Sam Mendes
----------------


In [ ]:
df.iloc[0].combined_features
df.combined_features.iloc[0]

'Action Adventure Fantasy Science Fiction culture clash future space war space colony society Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez James Cameron'

# **Process**

In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.combined_features)

In [ ]:
len(cv.get_feature_names_out())

14845

In [ ]:
count_matrix.shape

(4803, 14845)

In [ ]:
print(cv.get_feature_names_out()[0:10])
print(cv.get_feature_names_out()[0:-10])


['11' '15th' '17th' '18th' '1910s' '1917' '1920s' '1930s' '1950s' '1960s']
['11' '15th' '17th' ... 'zubaida' 'zucker' 'zuckerman']


In [ ]:
print(count_matrix.toarray()[:10, 10:30])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
cosine_sim = cosine_similarity(count_matrix)
cosine_sim.shape

(4803, 4803)

In [ ]:
print(cosine_sim[:2,:8].round(2))

[[1.   0.11 0.12 0.04 0.23 0.19 0.   0.14]
 [0.11 1.   0.08 0.04 0.07 0.26 0.   0.07]]


# **Recommend**

In [ ]:
user_choice = 'Avatar'
ref_index = df[df.title.str.contains(user_choice, case=False)].index[0]

print('user choice ref_index = {}'.format(ref_index))

user choice ref_index = 0


In [ ]:
similar_movies = list(enumerate(cosine_sim[ref_index]))
similar_movies[:8]

[(0, 1.0000000000000004),
 (1, 0.10540925533894599),
 (2, 0.12038585308576921),
 (3, 0.03774256780481986),
 (4, 0.23094010767585033),
 (5, 0.1924500897298753),
 (6, 0.0),
 (7, 0.1405456737852613)]

In [ ]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]
sorted_similar_movies[:6]


[(94, 0.42339019740572564),
 (2403, 0.3774256780481986),
 (3208, 0.3464101615137755),
 (47, 0.34426518632954817),
 (56, 0.33596842045264647),
 (3158, 0.33333333333333337)]

In [ ]:
df.title[94]

'Guardians of the Galaxy'

In [ ]:
from sqlalchemy.sql.base import elements
print('Recommend movies for [' + user_choice+']')
print('-----------------------')
for i,element in enumerate(sorted_similar_movies):
  similar_movies_id = element[0]
  similar_movies_title = df.title.iloc[similar_movies_id]
  s_score = element[1]
  print('{:30} -> {:.3f}'.format(similar_movies_title,s_score))

  if i> 6:
    break

Recommend movies for [Avatar]
-----------------------
Guardians of the Galaxy        -> 0.423
Aliens                         -> 0.377
Star Wars: Clone Wars: Volume 1 -> 0.346
Star Trek Into Darkness        -> 0.344
Star Trek Beyond               -> 0.336
Alien                          -> 0.333
Lockout                        -> 0.314
Jason X                        -> 0.308
